# test note


* jupyterはコンテナ起動すること
* テストベッド一式起動済みであること


In [1]:
!pip install --upgrade pip
!pip install --force-reinstall ../lib/ait_sdk-0.1.5-py3-none-any.whl

     |████████████████████████████████| 1.5 MB 5.8 MB/s eta 0:00:01     |████▌                           | 215 kB 5.8 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.3.1
    Uninstalling pip-20.3.1:
      Successfully uninstalled pip-20.3.1
Processing /workdir/root/lib/ait_sdk-0.1.5-py3-none-any.whl
     |████████████████████████████████| 95 kB 4.2 MB/s eta 0:00:011
  Using cached nbconvert-6.0.7-py3-none-any.whl (552 kB)
     |████████████████████████████████| 14.9 MB 9.3 MB/s eta 0:00:01    |█████████████                   | 6.0 MB 9.0 MB/s eta 0:00:01     |██████████████████████████▌     | 12.3 MB 9.3 MB/s eta 0:00:01
     |████████████████████████████████| 465 kB 11.4 MB/s eta 0:00:01
  Using cached nbformat-5.0.8-py3-none-any.whl (172 kB)
     |████████████████████████████████| 4.0 MB 5.5 MB/s eta 0:00:01     |██████████████▋                 | 1.8 MB 5.5 MB/s eta 0:00:01     |███████████████████▏            | 2.4 MB 5.5 MB/s eta 0:00:01
     |█

In [2]:
from pathlib import Path
import json

# settings cell

ait_name='eval_regression_analyze_coefficient_determination_tf2.3'
ait_full_name='eval_regression_analyze_coefficient_determination_tf2.3_0.1'

# mounted dir
ait_dir = Path(f'/workdir/root/ait/{ait_full_name}')

# (dockerホスト側の)インベントリ登録用アセット格納ルートフォルダ
current_dir = %pwd
with open(f'{current_dir}/config.json', encoding='utf-8') as f:
    json_ = json.load(f)
    root_dir = json_['host_ait_root_dir']
    is_container = json_['is_container']
invenotory_root_dir = f'{root_dir}\\ait\\{ait_full_name}\\local_qai\\inventory'

# entry point address
# コンテナ起動かどうかでポート番号が変わるため、切り替える
if is_container:
    backend_entry_point = 'http://host.docker.internal:8888/qai-testbed/api/0.0.1'
    ip_entry_point = 'http://host.docker.internal:8888/qai-ip/api/0.0.1'
else:
    backend_entry_point = 'http://host.docker.internal:5000/qai-testbed/api/0.0.1'
    ip_entry_point = 'http://host.docker.internal:6000/qai-ip/api/0.0.1'

# aitのデプロイフラグ
# 一度実施すれば、それ以降は実施しなくてＯＫ
is_init_ait = True

# インベントリの登録フラグ
# 一度実施すれば、それ以降は実施しなくてＯＫ
is_init_inventory = True


In [3]:
import requests
from pathlib import Path
import zipfile
from glob import glob
import pprint
import json
import time

def get_bk(path:str, is_print_response:bool = True, is_print_json:bool = True):
    response = requests.get(f'{backend_entry_point}{path}')
    if is_print_response:
        print(response)
    if is_print_json:
        pprint.pprint(response.json())
    return response

def get_ip(path:str, is_print_response:bool = True, is_print_json:bool = True):
    response = requests.get(f'{ip_entry_point}{path}')
    if is_print_response:
        print(response)
    if is_print_json:
        pprint.pprint(response.json())
    return response

def post_bk(path:str, data):
    response = requests.post(f'{backend_entry_point}{path}', 
                             json.dumps(data),
                             headers={'Content-Type': 'application/json'}) 
    print(response)  
    pprint.pprint(response.json())
    return response

def post_ip(path:str, data):
    response = requests.post(f'{ip_entry_point}{path}', 
                             json.dumps(data),
                             headers={'Content-Type': 'application/json'}) 
    print(response)  
    pprint.pprint(response.json())
    return response
    
def uploadzip_id(zip_path:str):
    fileobj = open(zip_path, 'rb')
    response = requests.post(f'{ip_entry_point}/deploy-dag', 
                             data = {"name":"dag_zip"}, 
                             files={"archive": (str(Path(zip_path).name), fileobj)})
    print(response)
    pprint.pprint(response.json())
    return response

def create_zip():
    zip_dir = ait_dir
    zip_file = zip_dir.parent / f'{ait_full_name}.zip'

    with zipfile.ZipFile(zip_file, 'w', compression=zipfile.ZIP_STORED) as new_zip:
        new_zip.write(str(zip_dir.joinpath('dag.py')), arcname=f'{ait_full_name}/dag.py')
        new_zip.write(str(zip_dir.joinpath('dev').joinpath('dockerfile')), arcname=f'{ait_full_name}/dev/dockerfile')
        
        dir_path = zip_dir.joinpath('dev').joinpath('repository')
        files = glob(str(dir_path.joinpath('**')), recursive=True)
        for file in files:
            file_name = Path(file).name
            new_zip.write(file, arcname=f'{ait_full_name}/dev/repository/{file_name}')
    return zip_file

def async_build_ait(zip_path):
    zip_name = str(Path(zip_path).name)
    with open(zip_path, 'rb') as file_obj:
        response = requests.post(f'{ip_entry_point}/async-deploy-dag', 
                                 files={"dag_zip": (zip_name, file_obj)})
    print(response)
    pprint.pprint(response.json())
    return response
    
def post_manifest(manifest_path):
    manifest_name = str(Path(manifest_path).name)
    with open(manifest_path, 'rb') as file_obj:
        response = requests.post(f'{backend_entry_point}/testRunners/manifest', 
                                 files={"ait.manifest": (manifest_name, file_obj)})
    print(response)
    pprint.pprint(response.json())
    return response

def _find_file(dir_path: Path, file_name: str):
    files = glob(str(dir_path.joinpath(f'**/{file_name}')), recursive=True)
    if len(files) == 0:
        raise Exception(f'not found {file_name} in zip.')
    elif len(files) > 1:
        raise Exception(f'{file_name} must be one exists in zip.')
    return files[0]

def deploy_ait_async(need_build_container:bool=True):    
    # add manifest
    post_manifest(_find_file(ait_dir, 'ait.manifest.json'))

    # deploy
    if need_build_container:
        zip_file = create_zip()
        async_build_ait(zip_file)

def wait_deploy_complete(wait_time:float=1):
    # animation = "|/-\\"
    animation = [
        " [*     ]",
        " [ *    ]",
        " [  *   ]",
        " [   *  ]",
        " [    * ]",
        " [     *]",
        " [    * ]",
        " [   *  ]",
        " [  *   ]",
        " [ *    ]",
    ]
    i = 0

    while True:
        print(animation[i % len(animation)], end="\r")
        i += 1

        json = get_ip('/async-deploy-dag', is_print_response=False, is_print_json=False).json()
        if json['Code'] != 'D00011':
            if json['Code'] != 'D00010':
                # D00010以外はエラー発生のため、コンソールに出力
                pprint.pprint(json)
            else:
                print('complete')
            break
        time.sleep(wait_time)

def wait_run_complete(wait_time:float=1):
    # animation = "|/-\\"
    animation = [
        " [*     ]",
        " [ *    ]",
        " [  *   ]",
        " [   *  ]",
        " [    * ]",
        " [     *]",
        " [    * ]",
        " [   *  ]",
        " [  *   ]",
        " [ *    ]",
    ]
    i = 0

    while True:
        print(animation[i % len(animation)], end="\r")
        i += 1

        run_status = get_bk('/dep-a/mlComponents/1/testDescriotions/run-status', is_print_response=False, is_print_json=False).json()
        if run_status['Job']['Status'] != 'RUNNING':
            pprint.pprint(run_status['Runs'])
            break
        time.sleep(wait_time)


In [4]:
from ait_sdk.test.hepler import Helper
helper = Helper(backend_entry_point=backend_entry_point, 
                ip_entry_point=ip_entry_point,
                ait_dir=ait_dir,
                ait_full_name=ait_full_name)

In [5]:
# health check

get_bk('/health-check')
get_ip('/health-check')

<Response [200]>
{'Code': 0, 'Message': 'alive.'}
<Response [200]>
{'Code': 0, 'Message': 'alive.'}


<Response [200]>

In [6]:
# create ml-component
res = helper.post_ml_component(name=f'MLComponent_{ait_full_name}', description=f'Description of {ait_full_name}', problem_domain=f'ProbremDomain of {ait_full_name}')
helper.set_ml_component_id(res['MLComponentId'])
ml_component_id = res['MLComponentId']

<Response [200]>
{'MLComponentId': 6,
 'Result': {'Code': 'P22000', 'Message': 'add ml-component success.'}}


In [7]:
# deploy AIT
if is_init_ait:
#     deploy_ait_async(need_build_container=True)
#     wait_deploy_complete(wait_time=0.5)
    helper.deploy_ait_non_build()
else:
    print('skip deploy AIT')

<Response [400]>
{'Code': 'T54000',
 'Message': 'already exist ait = '
            'eval_regression_analyze_coefficient_determination_tf2.3-0.1'}
<Response [200]>
{'Code': 'D00001', 'Message': 'Deploy success'}


In [8]:
res = helper.get_data_types()
model_data_type_id = [d for d in res['DataTypes'] if d['Name'] == 'model'][0]['Id']
dataset_data_type_id = [d for d in res['DataTypes'] if d['Name'] == 'dataset'][0]['Id']
res = helper.get_file_systems()
unix_file_system_id = [f for f in res['FileSystems'] if f['Name'] == 'UNIX_FILE_SYSTEM'][0]['Id']
windows_file_system_id = [f for f in res['FileSystems'] if f['Name'] == 'WINDOWS_FILE'][0]['Id']

In [9]:
# add inventories

if is_init_inventory:
    post_bk(f'/dep-a/mlComponents/{ml_component_id}/inventories', {
        'Name': 'train_data',
        'TypeId': dataset_data_type_id, 'FileSystemId': windows_file_system_id,
        'FilePath': f'{invenotory_root_dir}\\dataset_for_verification\\train_data.csv',
        'Description': "幸福度調査データで使用した学習用データ",
        'Formats': ["csv"],
    })
    post_bk(f'/dep-a/mlComponents/{ml_component_id}/inventories', {
        'Name': 'model_1',
        'TypeId': model_data_type_id, 'FileSystemId': windows_file_system_id,
        'FilePath': f'{invenotory_root_dir}\\trained_model\\model_1.h5',
        'Description': "幸福度調査データで構築した幸福度予測モデル",
        'Formats': ["h5"],
    })
else:
    print('skip add inventories')

<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}
<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}


In [10]:
# get ait_json and inventory_jsons

res_json = get_bk('/QualityMeasurements/RelationalOperators', is_print_json=False).json()
eq_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '=='][0])
nq_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '!='][0])
gt_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '>'][0])
ge_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '>='][0])
lt_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '<'][0])
le_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '<='][0])

res_json = get_bk('/testRunners', is_print_json=False).json()
ait_json = [j for j in res_json['TestRunners'] if j['Name'] == ait_name][-1]

res_json = get_bk(f'/dep-a/mlComponents/{ml_component_id}/inventories', is_print_json=False).json()
inv_verification_json = [j for j in res_json['Inventories'] if j['Name'] == 'train_data'][-1]
inv_model_json = [j for j in res_json['Inventories'] if j['Name'] == 'model_1'][-1]

<Response [200]>
<Response [200]>
<Response [200]>


In [11]:
# add teast_descriptions

post_bk(f'/dep-a/mlComponents/{ml_component_id}/testDescriotions', {
    "Name": '{}_Happiness_Score'.format(ait_json['Name']),
    "QualityDimensionID": ait_json['QualityDimensionId'],
    "QualityMeasurements": [
        {"Id":ait_json['Report']['Measures'][0]['Id'], "Value":"0.5", "RelationalOperatorId":gt_id, "Enable":True}
    ],
    "TargetInventories": [
        {"Id":1, "InventoryId": inv_model_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][0]['Id']},
        {"Id":2, "InventoryId": inv_verification_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][1]['Id']}
    ],
    "TestRunner": {
        "Id":ait_json['Id'],
        "Params":[
            {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][0]['Id'], "Value":"Happiness Score"}
        ]
    }
})

<Response [200]>
{'Result': {'Code': 'T22000', 'Message': 'append test description success.'}}


<Response [200]>

In [12]:
# get test_description_jsons

res_json = get_bk(f'/dep-a/mlComponents/{ml_component_id}/testDescriotions', is_print_json=False).json()
td_1_json = [j for j in res_json['Test']['TestDescriptions'] if j['Name'] == '{}_Happiness_Score'.format(ait_json['Name'])][-1]


<Response [200]>


In [13]:
# run test_descriptions

helper.post_run_and_wait(td_1_json['Id'])

<Response [200]>
{'Job': {'Id': '5', 'StartDateTime': '2021-01-26 16:41:07.808328+09:00'},
 'Result': {'Code': 'R12000', 'Message': 'job launch success.'}}
[{'Id': 5,
  'Result': 'OK',
  'ResultDetail': 'degree_of_freedom_adjusted_coefficient_determination : '
                  'OK.\n',
  'Status': 'DONE',
  'TestDescriptionID': 6}]


In [14]:
res_json = get_bk('/dep-a/mlComponents/{}/testDescriotions/{}'.format(ml_component_id, td_1_json['Id']), is_print_json=False).json()
pprint.pprint(res_json)

<Response [200]>
{'Result': {'Code': 'T32000', 'Message': 'get detail success.'},
 'TestDescriptionDetail': {'Id': 6,
                           'Name': 'eval_regression_analyze_coefficient_determination_tf2.3_Happiness_Score',
                           'Opinion': '',
                           'QualityDimension': {'Id': 5,
                                                'Name': 'Accuracy_of_trained_model'},
                           'QualityMeasurements': [{'Description': '0~1の値をとり、1に近いほど精度の高い予測ができているといえる。',
                                                    'Enable': True,
                                                    'Id': 34,
                                                    'Name': 'degree_of_freedom_adjusted_coefficient_determination',
                                                    'RelationalOperatorId': 3,
                                                    'Structure': 'single',
                                                    'Value': '0.5'}],
             

In [15]:
# generate report
# res = post_bk(f'/dep-a/mlComponents/{ml_component_id}/testDescriotions/reportGenerator', {
#     "Command": "Generate",
#     "Destination": [str(td_1_json['Id'])]
# })
res = helper.post_report(td_1_json['Id'])

<Response [200]>
{'OutParams': {'ReportUrl': 'http://127.0.0.1:8888/qai-testbed/api/0.0.1/download/238'},
 'Result': {'Code': 'D12000', 'Message': 'command invoke success.'}}
